<a href="https://colab.research.google.com/github/adarsh415/gpt-pytorch/blob/main/gpt_Residual_Norm_Block_FeedForward_multiHead_attention.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2023-07-23 03:49:53--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.05s   

2023-07-23 03:49:53 (20.5 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [2]:
# read it to inspect it
with open('input.txt', 'r', encoding='utf-8') as f:
  text = f.read()

In [3]:
print("length of dataset in characters: ", len(text))

length of dataset in characters:  1115394


In [4]:
#let's look at the first 1000 characters
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [5]:
# here are all unique characters that occurs in text
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [6]:
# create mapping from characters to integers
stoi = {ch:i for i, ch in enumerate(chars)}
itos = {i:ch for i, ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s] # encoder: takes a string, returns sequence of intgers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: takes list of integers, outputs string

print(encode('hii there'))
print(decode(encode('hii there')))

[46, 47, 47, 1, 58, 46, 43, 56, 43]
hii there


In [7]:
import torch
from torch import nn
import torch.nn.functional as F

In [8]:
batch_size=64 # how many independent sequences will process in parallel
block_size=256 # what is the maximum context length of predictions?
n_embed = 384
n_layer = 6
n_head = 6
dropout = 0.2
lr_rate = 3e-4
eval_interval = 500
eval_iters = 200
max_iters = 5000
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [9]:
data = torch.tensor(encode(text), dtype=torch.long, device=device )
# print(data.shape, data.dtype)
# print(data[:1000])

In [10]:
#Let's split dataset into train and validation set
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

In [11]:
# train_data[:block_size + 1]

In [12]:
# x = train_data[:block_size]
# y = train_data[1:block_size+1]
# for t in range(block_size):
#   context = x[:t+1]
#   target = y[t]
#   print(f"when context is: {context} target is: {target}")

In [13]:
torch.manual_seed(1023)
def get_batch(split):
  data = train_data if split=='train' else val_data
  ix = torch.randint(len(data) - block_size, (batch_size,))
  x = torch.stack([data[i:i+block_size] for i in ix])
  y = torch.stack([data[i+1:i+block_size+1] for i in ix])
  return x.to(device), y.to(device)

# xb, yb = get_batch('train')
# print('inputs')
# print(xb.shape)
# print(xb)
# print('target')
# print(yb.shape)
# print(yb)

# print('---------')

# for b in range(batch_size): # batch dimension
#   for t in range(block_size): # time dimension
#     context = xb[b, :t+1]
#     target = yb[b, t]
#     print(f"when context is: {context.tolist()} target is: {target}")


In [14]:
class Head(nn.Module):
  def __init__(self, head_size):
    super().__init__()
    self.key = nn.Linear(n_embed, head_size, bias=False)
    self.query = nn.Linear(n_embed, head_size, bias=False)
    self.value = nn.Linear(n_embed, head_size, bias=False)
    self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

    self.dropout = nn.Dropout(dropout)

  def forward(self, x):
    B, T, C = x.shape
    k = self.key(x)
    q = self.query(x)
    # compute the attention scores ('affinities)
    wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) ---> (B, T, T) , used scaled attention here
    wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T) , this is part of decoder so does not communicate with past
    wei = F.softmax(wei, dim=-1) # (B, T, T)
    wei = self.dropout(wei)
    # perform the weighted aggregation of values
    v = self.value(x) # (B, T, C)
    out = wei @ v # (B, T, T) @ (B, T, C) --> (B, T, C)
    return out

In [37]:
class MultiHeadAttention(nn.Module):
  """Multi head attention"""
  def __init__(self, num_heads, head_size):
    super().__init__()
    self.head = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
    self.proj = nn.Linear(head_size*num_heads, n_embed) #project layer

    self.dropout = nn.Dropout(dropout)

  def forward(self, x):
    out = torch.cat([h(x) for h in self.head], dim=-1) # concatenate along the channel dimension, i.e last dimension
    out = self.proj(out)
    out = self.dropout(out)
    return out

In [38]:
class FeedForward(nn.Module):
  """Simple linear layer followed by non linearilty"""

  def __init__(self, n_embed):
    super().__init__()
    self.net = nn.Sequential(
        nn.Linear(n_embed, 4 * n_embed),
        nn.ReLU(),
        nn.Linear(4 * n_embed, n_embed),
        nn.Dropout(dropout)
    )
  def forward(self, x):
    return self.net(x)

In [39]:
class Block(nn.Module):
  """ Transformer block: communication followed by computation """

  def __init__(self, n_embed, n_head):
    super().__init__()
    # n_embed: embedding dimension, n_head: the number of heads we'd like
    head_size = n_embed // n_head
    self.sa = MultiHeadAttention(n_head, head_size)
    self.ffwd = FeedForward(n_embed)
    self.l1 = nn.LayerNorm(n_embed)
    self.l2 = nn.LayerNorm(n_embed)

  def forward(self, x):
    x = x + self.sa(self.l1(x)) # residual block
    # x = self.l1(x)
    x = x + self.ffwd(self.l2(x)) # residual block
    # x = self.l2(x)
    return x


In [40]:
torch.manual_seed(1033)

class BigramLanguageModel(nn.Module):
  def __init__(self):
    super(BigramLanguageModel, self).__init__()
    self.token_embedding_table = nn.Embedding(vocab_size, n_embed)
    self.position_embedding_table = nn.Embedding(block_size, n_embed)
    self.blocks = nn.Sequential(*[Block(n_embed, n_head=n_head) for _ in range(n_layer)])
    self.ln_f = nn.LayerNorm(n_embed) # final layer norm
    self.lm_head = nn.Linear(n_embed, vocab_size)

  def forward(self, idx, target=None):
    B, T = idx.shape

    # idx and targets are both (B,T) tensors of integer
    token_embed = self.token_embedding_table(idx) # (B,T,C) where C is n_embed
    pos_embed = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
    X = token_embed + pos_embed # (B, T, C)
    X = self.blocks(X) #(B, T, C)
    X = self.ln_f(X) #(B,T,C)
    logits = self.lm_head(X) # (B, T, Vocab_size)

    if target is not None:
      B, T, C = logits.shape
      logits = logits.view(B*T, C)
      target = target.view(B*T) # or target.view(-1)
      loss = F.cross_entropy(logits, target)

    else:
      loss = None

    return logits, loss

  def generate(self, idx, max_new_char):
    #idx is (B, T) array of indices in current context
    for _ in range(max_new_char):
      # print(idx.shape)
      # print(idx)
      # crop idx to the last block_size tokens
      idx_cond = idx[:, -block_size:]
      # print(idx_cond)
      # get the predictions
      logits, loss = self(idx_cond)
      # Focus only on last time step
      logits = logits[:, -1, :] # becomes [B, C]
      # apply softmax to get probabilities
      probs = F.softmax(logits, dim=-1) # (B,C)
      # sample from distribution
      idx_next = torch.multinomial(probs, num_samples=1) #(B, 1)
      # append sampled index to running index
      idx = torch.cat((idx, idx_next), dim=1) # (b , T+1)

    return idx

model = BigramLanguageModel()
m = model.to(device)
# logits, loss = m(xb, yb)
# print(logits.shape)
# print(loss)
# start_token = torch.zeros((1,1), dtype=torch.long)
# print(decode(m.generate(start_token, 100)[0].tolist()))

In [41]:
# create Optimizer
optimizer = torch.optim.Adam(m.parameters(), lr = lr_rate)

In [42]:
@torch.no_grad()
def estimate_loss():
  out = {}
  m.eval()
  for split in ['train', 'val']:
    losses = torch.zeros(eval_iters, device=device)
    for k in range(eval_iters):
      xb, yb = get_batch(split)
      logits, loss = m(xb, yb)
      losses[k] = loss.item()
    out[split] = losses.mean()
  m.train()
  return out

In [43]:
for steps in range(max_iters):

  if steps % eval_interval == 0:
    losses = estimate_loss()
    print(f"step {steps}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

  xb, yb = get_batch('train')
  logits, loss = m(xb, yb)
  optimizer.zero_grad(set_to_none=True)
  loss.backward()
  optimizer.step()

print(loss.item())

step 0: train loss 4.3600, val loss 4.3610
step 500: train loss 2.0167, val loss 2.1013
step 1000: train loss 1.6072, val loss 1.7750
step 1500: train loss 1.4429, val loss 1.6453
step 2000: train loss 1.3496, val loss 1.5708
step 2500: train loss 1.2813, val loss 1.5358
step 3000: train loss 1.2313, val loss 1.5054
step 3500: train loss 1.1906, val loss 1.5010
step 4000: train loss 1.1506, val loss 1.4911
step 4500: train loss 1.1153, val loss 1.4790
1.2000312805175781


In [44]:
start_token = torch.zeros((1,1), dtype=torch.long, device=device)
print(decode(m.generate(start_token, 500)[0].tolist()))


Of Clording-lawys and soldiers at this?
Off and hope, piece, rood, Your wounds it, the deable,
pale live, no white York requested how:
Lo, believe-venom, who could I cannot make
Your friendly exulcure by his plotteness mast
I' the degre and night, am I to go,
To Barry Bona, forbear, a toice impound,
It will I mother in some back orior to beams.

VIRGILIA:
Dorank not have either, tare in his country.
I can make it well.

LARTIUS:
I have passed on my brother's bare
Is cruel things; but come, to wa
